# 호텔 예약 취소 예측

## 서론

호텔 방을 예약하기에 가장 좋은 시기가 언제인지 궁금한 적이 있습니까? 
아니면 최고의 일일 요금을 얻기 위해 최적의 체류 기간? 
호텔이 불균형 적으로 많은 수의 특별 요청을 받을 가능성이 있는지 여부를 예측하려면 어떻게 해야 합니까?

이 호텔 예약 데이터 세트는 이러한 질문을 탐색하는 데 도움이 됩니다!

[Hotel booking demand](https://www.kaggle.com/jessemostipak/hotel-booking-demand)

### 변수 설명

 - 호텔 : 호텔 구분(H1 = 리조트 호텔, H2 = 시티 호텔)
 - is_canceled : 예약이 취소되었는지 여부를 나타내는 값(0:숙박, 1:취소)
 - lead_time : 예약 시작 날짜와 도착 날짜 사이에 경과 된 일수
 - arrival_date_year : 도착 연도
 - arrival_date_month : 
 - arrival_date_week_number	: 
 - arrival_date_day_of_month : 
 - stays_in_weekend_nights :
 - stays_in_week_nights :
 - 
 
 - customer_type
     - Transient : 그외 채널로 호텔 객실을 예약하는 개별 고객
     - Contract : 항공사 크루나 장기 투숙고객 등
     - Transient-Party : 써드파티 채널을 통해 호텔 객실을 예약하는 개별 고객
     - Group : 보통 10개 이상의 객실을 한꺼번에 이용하는 고객

## 라이브러리 로드

In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [30]:
# 한글폰트를 설정해 주지 않으면 그래프 상에서 한글이 깨져보입니다.
# 한글이 출력될 수 있도록 폰트 설정을 해줍니다.
plt.rc('font', family='AppleGothic')

# 윈도우, 맥 외의 OS는 별도로 설정해 주세요.
# 마이너스 폰트 깨지는 문제에 대한 대처
plt.rc('axes', unicode_minus=False)

In [31]:
# 레티나 설정을 해주면 글씨가 좀 더 선명하게 보입니다.
# 폰트의 주변이 흐릿하게 보이는 것을 방지합니다.
from IPython.display import set_matplotlib_formats

set_matplotlib_formats('retina')

## 데이터셋 가져오기

In [45]:
df = pd.read_csv('data/hotel_bookings.csv')
df.shape

(119390, 32)

In [46]:
# 컬럼이 모두 보이도록 설정합니다.

pd.options.display.max_columns = 100

In [47]:
# 미리보기를 합니다.

df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [65]:
df['customer_type'].unique()

array(['Transient', 'Contract', 'Transient-Party', 'Group'], dtype=object)

## 데이터 전처리

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
hotel                             119390 non-null object
is_canceled                       119390 non-null int64
lead_time                         119390 non-null int64
arrival_date_year                 119390 non-null int64
arrival_date_month                119390 non-null object
arrival_date_week_number          119390 non-null int64
arrival_date_day_of_month         119390 non-null int64
stays_in_weekend_nights           119390 non-null int64
stays_in_week_nights              119390 non-null int64
adults                            119390 non-null int64
children                          119386 non-null float64
babies                            119390 non-null int64
meal                              119390 non-null object
country                           119390 non-null object
market_segment                    119390 non-null object
distribution_channel              119390 n

### 결측치 채우기

In [49]:
df.isnull().sum().sort_values(ascending=False).head(15)

company                      112593
agent                         16340
country                         488
children                          4
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
is_canceled                       0
market_segment                    0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
babies                            0
dtype: int64

In [52]:
# 'country' 값의 결측치를 "None" 으로 채웁니다.
df['country'] = df['country'].fillna("None")
df

,hotel,arrival_date_month,meal,country,market_segment,distribution_channel,reserved_room_type,assigned_room_type,deposit_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,July,BB,PRT,Direct,Direct,C,C,No Deposit,Transient,Check-Out,2015-07-01
1,Resort Hotel,July,BB,PRT,Direct,Direct,C,C,No Deposit,Transient,Check-Out,2015-07-01
2,Resort Hotel,July,BB,GBR,Direct,Direct,A,C,No Deposit,Transient,Check-Out,2015-07-02
3,Resort Hotel,July,BB,GBR,Corporate,Corporate,A,A,No Deposit,Transient,Check-Out,2015-07-02
4,Resort Hotel,July,BB,GBR,Online TA,TA/TO,A,A,No Deposit,Transient,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,August,BB,BEL,Offline TA/TO,TA/TO,A,A,No Deposit,Transient,Check-Out,2017-09-06
119386,City Hotel,August,BB,FRA,Online TA,TA/TO,E,E,No Deposit,Transient,Check-Out,2017-09-07
119387,City Hotel,August,BB,DEU,Online TA,TA/TO,D,D,No Deposit,Transient,Check-Out,2017-09-07
119388,City Hotel,August,BB,GBR,Online TA,TA/TO,A,A,No Deposit,Transient,Check-Out,2017-09-07


In [ ]:
# df['company'] = df['company'].dropna(axis=1)
# df['agent'] = df['agent'].fillna(0)
# df['children'] = df['children'].fillna(0)
df

### 학습, 예측 데이터셋 만들기

In [36]:
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')

In [ ]:
# 학습, 예측에 사용할 컬럼

feature_names = []

In [ ]:
# 예측값
# label_name 이라는 변수에 예측할 컬럼의 이름을 담습니다.

label_name = 

In [ ]:
# 학습, 예측 데이터셋 만들기
X = df[feature_names]
X.shape

In [ ]:
y = df[label_name]
y.shape

### StandardScaler 로 데이터 스케일 맞추기

In [ ]:
from sklearn.preprocessing import StandardScaler

X_ = StandardScaler().fit(X).transform(X)
X_

In [ ]:
# 스케일링이 잘 되었는지 히스토그램으로 확인

h = pd.DataFrame(X_).hist(figsize=(15, 15))

In [ ]:
# model_selection을 통해 데이터를 나눕니다.

from sklearn.model_selection import train_test_split

# train_test_split?

X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2, random_state=42) 

In [ ]:
print(X_train.shape)
X_train[:1]

In [ ]:
print(X_test.shape)
X_test[:1]

### 학습세트와 테스트세트가 잘 만들어졌는지 확인¶

In [ ]:
print(X_train.shape)
X_train.head()

In [ ]:
print(X_test.shape)
X_test.head()

## EDA

### pandas-profiling

In [ ]:
# pandas_profiling 을 통해 ProfileReport 로 간단한 eda 리포트를 만들어 봅니다.

from pandas_profiling import ProfileReport

Profile = ProfileReport(X_train)
Profile.to_file(output_file = 'house_price.html')

### histogram

In [ ]:
h = df.hist(figsize=(15, 15))